In [1]:
import pandas as pd
import numpy as np

from bokeh.io import output_file, output_notebook,curdoc  
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource,HoverTool, CustomJS, Select,RangeSlider
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Tabs, Panel
from datetime import date

output_file('filename.html')  
fig = figure()

In [2]:
df_cov = pd.read_json("https://apicovid19indonesia-v2.vercel.app/api/indonesia/harian")
df_cov.to_csv("covid_harian.csv",index=False)
df_cov

,positif,dirawat,sembuh,meninggal,positif_kumulatif,dirawat_kumulatif,sembuh_kumulatif,meninggal_kumulatif,lastUpdate,tanggal
0,2,2,0,0,2,2,0,0,1583107200000,2020-03-02T00:00:00.000Z
1,0,0,0,0,2,2,0,0,1583193600000,2020-03-03T00:00:00.000Z
2,0,0,0,0,2,2,0,0,1583280000000,2020-03-04T00:00:00.000Z
3,0,0,0,0,2,2,0,0,1583366400000,2020-03-05T00:00:00.000Z
4,2,2,0,0,4,4,0,0,1583452800000,2020-03-06T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...
674,404,220,180,4,4264136,4878,4115149,144109,1641340800000,2022-01-05T00:00:00.000Z
675,533,317,209,7,4264669,5195,4115358,144116,1641427200000,2022-01-06T00:00:00.000Z
676,518,299,214,5,4265187,5494,4115572,144121,1641513600000,2022-01-07T00:00:00.000Z
677,479,298,175,6,4265666,5792,4115747,144127,1641600000000,2022-01-08T00:00:00.000Z


In [3]:
df_use = df_cov[["positif","sembuh","tanggal"]]
df_use['tanggal'] = df_use['tanggal'].str[:10]
df_use

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,positif,sembuh,tanggal
0,2,0,2020-03-02
1,0,0,2020-03-03
2,0,0,2020-03-04
3,0,0,2020-03-05
4,2,0,2020-03-06
...,...,...,...
674,404,180,2022-01-05
675,533,209,2022-01-06
676,518,214,2022-01-07
677,479,175,2022-01-08


In [4]:
source_df = ColumnDataSource(data=df_use)
# Output the visualization directly in the notebook
output_notebook()

# Create a figure plot
fig = figure(title='Covid',
             plot_height=400, plot_width=len(df_use),
             x_axis_label='Day Number', y_axis_label='Total',
             x_minor_ticks=2, y_range=(0, df_cov["positif"].max()*1.05),
             x_range=(0, len(df_use)-1),
             tools="box_zoom,reset,save")


fig.legend.location = 'top_left'

#tooltip hover
tooltips = [('Day', '@index'),
            ('Date', '@tanggal'),
            ('Sembuh', '@sembuh'),
            ('Positif', '@positif'),
           ]


#dropdown interactive

select = Select(title="Show:", value="Line 1", options=["All", "Positif", "Sembuh"])
select.js_on_change("value", CustomJS(args=dict(line_1=fig.vbar("index", 0,"positif",  source=source_df,color='red', width=3.75,legend_label='Positif'),
                                                line_2=fig.line("index", "sembuh", source=source_df, color='green', line_width=1, legend_label='Sembuh'),
                                                line_3=fig.line("index", "positif", source=source_df, color='red', line_width=1)), code="""

line_1.visible = true
line_2.visible = true
line_3.visible = true

if (this.value === "Positif") {
    line_2.visible = false 
} else if (this.value === "Sembuh"){
    line_1.visible = false
    line_3.visible = false
}else{
  line_1.visible = true
  line_2.visible = true
  line_3.visible = true
}
    
"""))


#slider day interactive
range_slider = RangeSlider(
    title="Day Range :",
    start=0,
    end=len(df_use),
    step=1,
    value=(fig.x_range.start, fig.x_range.end),
)
  
range_slider.js_link("value", fig.x_range, "start", attr_selector=0)
range_slider.js_link("value", fig.x_range, "end", attr_selector=1)
  

fig.add_tools( HoverTool(tooltips=tooltips,mode='vline' ))


layout = column(fig,select,range_slider)
show(layout)

/usr/local/lib/python3.7/dist-packages/bokeh/models/plots.py:766: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
